In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time
import requests
import os
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
from splinter import Browser
from selenium.webdriver.chrome.service import Service

my_service = Service(executable_path=r'C:\Users\pc\OneDrive\Desktop\TCG-scraper\chromedriver.exe')
browser = Browser('chrome', service=my_service)

In [3]:
# visiting the webpage

url = 'https://shop.tcgplayer.com/sellers'
browser.visit(url)

In [4]:
# xpath for the next button at the end of each page
# //*[@id="app"]/div/div/section[2]/section/section/section/div[2]/div[1]/a[2]/span/span/svg

# text_xpath = '//*[@id="SellerName"]'
seller = 'GAMELAND'
seller_list = [seller, 'xyz', 'abc']


# passing in the seller name

browser.fill('SellerName', seller)

In [5]:
browser.find_by_xpath('//*[@id="maincontentinnerpadding"]/div/div/form/div/ul/li[4]/div/input').click()
#time.sleep(3)
browser.find_by_xpath('//*[@id="maincontentinnerpadding"]/div/div[2]/div[3]/div/div[3]/button').click()
time.sleep(3)

In [6]:
price_list = []

In [7]:
#search-result__title
#inventory__price-with-shipping
#search-result__market-price--value


for i in range(3):
    req = browser.html
    soup = BeautifulSoup(req, "html.parser")

    cat_tags = soup.find_all('h3', class_ ="search-result__category-name")
    set_tags = soup.find_all('h4', class_ ="search-result__subtitle")
    title_tags = soup.find_all('span', class_ = "search-result__title")
    price1_tags = soup.find_all('span', class_ = "inventory__price-with-shipping")
    price2_tags = soup.find_all('span', class_ = "search-result__market-price--value")

    for  cat, sset, titles, price1, price2 in zip( cat_tags ,set_tags, title_tags, price1_tags, price2_tags):
        row = []
        row.append(cat.text)
        row.append(sset.text)
        row.append(titles.text)
        row.append(price1.text)
        row.append(price2.text)
        price_list.append(row)           
    
    browser.find_by_xpath('/html/body/div[2]/div/div/section[2]/section/section/section/div[2]/div[1]/a[2]').click()
    time.sleep(4)
    
    
                
               

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00CF72A3+45731]
	(No symbol) [0x00C82D51]
	(No symbol) [0x00B7880D]
	(No symbol) [0x00B5F75E]
	(No symbol) [0x00BCC11B]
	(No symbol) [0x00BDB2D3]
	(No symbol) [0x00BC7DD6]
	(No symbol) [0x00BA31F6]
	(No symbol) [0x00BA439D]
	GetHandleVerifier [0x01000716+3229462]
	GetHandleVerifier [0x010484C8+3523784]
	GetHandleVerifier [0x0104214C+3498316]
	GetHandleVerifier [0x00D81680+611968]
	(No symbol) [0x00C8CCCC]
	(No symbol) [0x00C88DF8]
	(No symbol) [0x00C88F1D]
	(No symbol) [0x00C7B2C7]
	BaseThreadInitThunk [0x7757FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77E67C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77E67C3E+238]


In [ ]:
browser.quit

In [ ]:
df = pd.DataFrame(price_list)
df

In [ ]:
df.columns = [  "category", "set", 'Name', 'Listed Price','Market Price']


In [ ]:
# reformat data


df["Market Price"] = df["Market Price"].str.replace("$", "")
df["Listed Price"] = df["Listed Price"].str.replace("$", "")
df['Market Price'] = df['Market Price'].astype(float)
df['Listed Price'] = df['Listed Price'].astype(float)



In [ ]:
df["over_under"] = (df["Listed Price"]) - (df['Market Price'])


In [ ]:
sorted_df = df.sort_values(["over_under"])
sorted_df.head(5)

In [ ]:
limit_df= sorted_df.loc[sorted_df['category'] == "Magic: The Gathering"]
limit_df= limit_df.iloc[:100]
limit_df

In [ ]:
limit_df.to_csv(f"{seller}_report.csv")